<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD-GA Analysis mode Tutorial

FAST-OAD-GA is an add-on package for [FAST-OAD framework](https://github.com/fast-aircraft-design/FAST-OAD) for performing rapid Overall Aircraft Design in the category General Aviation (GA). The computational core of FAST-OAD being based on the  [OpenMDAO framework](https://openmdao.org/).

This notebook will show you the basic step to perform an aircraft analysis based on core models.

To organize our work, we propose to use two user folders `data/` and `workdir/`. In `data/` we store the XML file which describes the aircraft to analyse, here the Beechcraft Duchess. In `workdir/`, we store files generated or modified by FAST-OAD-(GA).

In [1]:
import os.path as pth
import os
import openmdao.api as om
from fastoad import api as api_cs25
from fastga.command import api as api_cs23
import logging
from fastoad.gui import VariableViewer
import shutil

# Define relative path
DATA_FOLDER_PATH = 'data'
WORK_FOLDER_PATH = 'workdir'

# Final file names
AIRCRAFT1_FILE = pth.join(WORK_FOLDER_PATH, 'geometry_reference.xml')
AIRCRAFT2_FILE = pth.join(WORK_FOLDER_PATH, 'geometry_long_wing.xml')

Failed to import module fastga.models.performances.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.handling_qualities.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.aerodynamics.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.weight.cg.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.weight.mass_breakdown.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.geometry.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.aerodynamics.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.handling_qualities.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.load_analysis.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.loops.unitary_tests.test_wing_area.py
Failed to import module fastga.models.geometry.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.weight.cg.unita

In [2]:
# Clear work folder
shutil.rmtree(WORK_FOLDER_PATH, ignore_errors=True)
os.mkdir(WORK_FOLDER_PATH)

# For using all screen width
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:95% !important; }</style>"))

## 1. Read, modify and save different aircraft geometry configurations

In this paragraph we will first run the geometry module on the reference aircraft to derive primary parameters into the detailed geometry set.

**This is done in four steps:**
1. copy the [reference aircraft file](./data/reference_aircraft.xml) to workdir under geometry_inputs.xml name,
2. copy the [geometry.toml file](./data/geometry.toml) to workdir,
3. run process,
4. save output file under the specific name: geometry_reference.xml.

In [3]:
# Copy the reference geometry file (limited input parameters) as input file (name specified in .toml)
shutil.copy(pth.join(DATA_FOLDER_PATH, 'reference_aircraft.xml'), pth.join(WORK_FOLDER_PATH, 'geometry_inputs.xml'))

# Copy the .toml process file to the workdir
CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, 'geometry.yml')
shutil.copy(pth.join(DATA_FOLDER_PATH, 'geometry.yml'), CONFIGURATION_FILE)

# Launch an evaluation to obtain the output file (name specified in the .toml)
eval_problem = api_cs25.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

# Copy this file to a different name to avoid an overwritte when computing secong geometry
shutil.copy(pth.join(WORK_FOLDER_PATH, 'geometry_outputs.xml'), AIRCRAFT1_FILE)

# Open viewer
api_cs25.variable_viewer(AIRCRAFT1_FILE)

Error visiting fastga.models.aerodynamics.external.openvsp.compute_aero_slipstream_x57: Error installing bundle fastga.models.aerodynamics.external.openvsp.compute_aero_slipstream_x57: No module named 'fastga.models.aerodynamics.external.openvsp.compute_aero_slipstream_x57'
Error visiting fastga.models.aerodynamics.external.openvsp.compute_vn: Error installing bundle fastga.models.aerodynamics.external.openvsp.compute_vn: No module named 'fastga.models.aerodynamics.external.openvsp.compute_vn'
Error visiting fastga.models.aerodynamics.external.vlm.compute_vn: Error installing bundle fastga.models.aerodynamics.external.vlm.compute_vn: No module named 'fastga.models.aerodynamics.external.vlm.compute_vn'
Error visiting fastga.models.aerodynamics.lift_equilibrium: Error installing bundle fastga.models.aerodynamics.lift_equilibrium: No module named 'fastga.models.aerodynamics.lift_equilibrium'
Error visiting fastga.models.aerodynamics.unitary_tests.test_beechcraft_76: Error installing bundl

Failed to import module fastga.models.load_analysis.aerodynamic_loads_x57
Failed to import module fastga.models.load_analysis.aerostructural_loads_x57
Failed to import module fastga.models.weight.cg.unitary_tests.test_cirrus_sr22


**This process can be done using the api generate block analysis method to get this working such as a python function:**
1. Same as previous, copy the reference aircraft but already into the aircraft name (same file used all along the process: overwritten)
2. Import the Geometry module
3. Generate a block analysis based on this model

We are going to apply it to the second geometry.

In [4]:
from fastga.models.geometry.geometry import GeometryFixedTailDistance

# Copy reference aircraft file
shutil.copy(pth.join(DATA_FOLDER_PATH, 'reference_aircraft.xml'), AIRCRAFT2_FILE)

# Define the wing primary geometry parameters name as a list
var_inputs = ["data:geometry:wing:area", "data:geometry:wing:aspect_ratio", "data:geometry:wing:taper_ratio"]

# Declare function
compute_geometry = api_cs23.generate_block_analysis(
        GeometryFixedTailDistance(propulsion_id="fastga.wrapper.propulsion.basicIC_engine"),
        var_inputs,
        str(AIRCRAFT2_FILE),
        True,
    )

Consider we want to generate a second aircraft geometry modifying the wing by an increase of 15% of the spanwithout changing the first part of the wing.

**We have the following relationships:**

$virtualChord=\frac{area}{2*y_{2}+(y_{4}-y_{2})*(1+taperRatio)}$

$aspectRatio=\frac{span^2}{area}$

With $y_{2}$ the y-position of the root chord and $y_{4}$ the y-position of the tip chord.

**Meaning:**

$\frac{area}{area_{ref}}=\frac{2*y_{2}+(y_{4}-y_{2})*(1+taperRatio)}{2*y_{2,ref}+(y_{4,ref}-y_{2,ref})*(1+taperRatio_{ref})}\implies{}area=\frac{2*y_{2,ref}+(\lambda*y_{4,ref}-y_{2,ref})*(1+taperRatio)}{2*y_{2,ref}+(y_{4,ref}-y_{2,ref})*(1+taperRatio_{ref})}*area_{ref}$

With $\lambda$=1.15.

**With $y_{2}=0.599$, $y_{4}=6.182$ we can find the primary parameters to generate second aicraft:**

In [5]:
# Define reference aircraft values
span_old = 11.523
taper_ratio_old = 1.0
area_old = 16.81
y2_ref = 2.881
y4_ref = 5.7615

# Define functions
taper_func = lambda lamb: lamb * taper_ratio_old + (1 - lamb)
area_func = lambda taper_ratio_new, lamb : (2*y2_ref + (lamb*y4_ref-y2_ref)*(1+taper_ratio_new))/(2*y2_ref + (y4_ref-y2_ref)*(1+taper_ratio_old)) * area_old

# Calculate parameters
taper_ratio_new = taper_func(1.15)
area = area_func(taper_ratio_new, 1.15)

# Print results
print("area=" + str(area))
print("aspect_ratio=" + str((span_old*1.15)**2/area))
print("taper_ratio=" + str(taper_ratio_new))

area=19.3315
aspect_ratio=9.083667956573466
taper_ratio=1.0


Now we can use the previously created function to calculate the long wing aircraft.

In [8]:
# Compute geometry
inputs_dict = {"data:geometry:wing:area": (19.3315, "m**2"), "data:geometry:wing:aspect_ratio": (9.083, None), "data:geometry:wing:taper_ratio": (1.0, None)}
outputs_dict = compute_geometry(inputs_dict)

# Open viewer
api_cs25.variable_viewer(AIRCRAFT2_FILE)

We will finish by visualizing the two generated geometries before performing other calculation/analysis steps.

In [9]:
from fastoad.gui import aircraft_geometry_plot

fig = aircraft_geometry_plot(pth.join(WORK_FOLDER_PATH, 'geometry_reference.xml'), name='reference aircraft')
fig = aircraft_geometry_plot(pth.join(WORK_FOLDER_PATH, 'geometry_long_wing.xml'), name='long wing aircraft', fig=fig)
fig.show()

## 2. Performing aerodynamic analysis
In this chapter we will see how to perform a V-N analysis on both aricraft versions.

To do so, we will launch first the calculation of low-speed coefficients using the same technique as previously mentionned on geometry (on both aircraft files).

In [10]:
from fastga.models.aerodynamics.aerodynamics import Aerodynamics

# Declare function on 1st geometry file
compute_aero1 = api_cs23.generate_block_analysis(
        Aerodynamics(
            propulsion_id="fastga.wrapper.propulsion.basicIC_engine",
            use_openvsp=True,
            compute_mach_interpolation=True,
            compute_slipstream_low_speed=False,
            compute_slipstream_cruise=False,
        ),
        [],
        str(AIRCRAFT1_FILE),
        True,
    )


# Declare function on 2nd geometry file
compute_aero2 = api_cs23.generate_block_analysis(
        Aerodynamics(
            propulsion_id="fastga.wrapper.propulsion.basicIC_engine",
            use_openvsp=True,
            compute_mach_interpolation=True,
            compute_slipstream_low_speed=False,
            compute_slipstream_cruise=False,
        ),
        [],
        str(AIRCRAFT2_FILE),
        True,
    )

In [9]:
# Compute both aircraft designs
result1 = compute_aero1({})
result2 = compute_aero2({})

# Open viewer
api_cs25.variable_viewer(AIRCRAFT2_FILE)

Chord ratio value outside of the range in Roskam's book, value clipped
Chord ratio value outside of the range in Roskam's book, value clipped
Chord ratio value outside of the range in Roskam's book, value clipped
Chord ratio value outside of the range in Roskam's book, value clipped
Chord ratio value outside of the range in Roskam's book, value clipped
Chord ratio value outside of the range in Roskam's book, value clipped
Chord ratio value outside of the range in Roskam's book, value clipped
Chord ratio value outside of the range in Roskam's book, value clipped


Then, we will compute the V-N diagram point for both aircraft.

In [13]:
from fastga.models.aerodynamics.components.compute_vn import ComputeVNAndVH

# Define the design mass speed and altitude taken as inputs for V-N diagram
var_inputs = ["data:weight:aircraft:MTOW", "data:TLAR:v_cruise", "data:mission:sizing:main_route:cruise:altitude"]

# Declare function on 1st geometry file
compute_geometry1 = api_cs23.generate_block_analysis(
        ComputeVNAndVH(
            propulsion_id="fastga.wrapper.propulsion.basicIC_engine",
        ),
        var_inputs,
        str(AIRCRAFT1_FILE),
        True,
    )

# Declare function on 2nd geometry file
compute_geometry2 = api_cs23.generate_block_analysis(
        ComputeVNAndVH(
            propulsion_id="fastga.wrapper.propulsion.basicIC_engine",
        ),
        var_inputs,
        str(AIRCRAFT2_FILE),
        True,
    )

In [14]:
# Compute both aircraft designs using similar input parameters
inputs_dict = {"data:weight:aircraft:MTOW": (1700.0, "kg"), "data:TLAR:v_cruise": (158, "kn"), "data:mission:sizing:main_route:cruise:altitude": (8000.0, "ft")}

result1 = compute_geometry1(inputs_dict)
result2 = compute_geometry2(inputs_dict)

Now, we are gona plot the results for both aicrafts.

In [15]:
from fastga.utils.postprocessing.analysis_and_plots import evolution_diagram

fig = evolution_diagram(pth.join(WORK_FOLDER_PATH, 'geometry_reference.xml'), name='reference aircraft')
# fig = evolution_diagram(pth.join(WORK_FOLDER_PATH, 'geometry_long_wing.xml'), name='long wing aircraft', fig=fig)
fig.show()